In [12]:
import requests
#pip install langchain openai requests
#from langchain.prompts import PromptTemplate
#from langchain.chains import SequentialChain, LLMChain
#from langchain.llms import OpenAI
#from langchain.agents import Tool, initialize_agent
#import requests


# Step 1: Create a dictionary for soil-crop compatibility
def get_soil_crop_mapping():
    return {
        "black soil": ["wheat", "rice", "jowar", "bajra", "cotton", "sugarcane"],
        "laterite soil": ["pulses", "onion"],
        "alluvial soil": ["rice", "wheat"],
        "red soil": ["rice"],
        "marshy soil": ["sugarcane"],
    }

# Step 2: Validate soil-crop compatibility
def validate_soil_crop(soil_type, crop_type, soil_crop_mapping):
    soil_type = soil_type.strip().lower()
    crop_type = crop_type.strip().lower()

    if soil_type not in soil_crop_mapping:
        return False, [], f"The soil type '{soil_type}' is not valid. Please provide a valid soil type."

    suitable_crops = soil_crop_mapping.get(soil_type, [])
    if crop_type in suitable_crops:
        return True, [], f"The crop '{crop_type}' is suitable for {soil_type}. Here are some other crops suitable for {soil_type}: {', '.join(suitable_crops)}."
    else:
        return False, suitable_crops, f"The crop '{crop_type}' is not suitable for {soil_type}. However, here are some crops suitable for {soil_type}: {', '.join(suitable_crops)}."

# Step 3: Fetch real-time weather data
def get_weather_data(location):
    api_key = "db25e6c2b3448b5d3a1a14a30a4dc04e"  # Replace with your OpenWeatherMap API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            weather = {
                "temperature": data["main"]["temp"],
                "description": data["weather"][0]["description"],
                "rain": "rain" in data
            }
            return weather
        else:
            return {"error": f"Failed to fetch weather data for {location}. Please check the location or try again later."}
    except Exception as e:
        return {"error": str(e)}

# Step 4: Analyze weather data and provide advice
def analyze_weather(weather):
    if "error" in weather:
        return weather["error"]
    if weather["rain"]:
        return f"The weather is rainy with '{weather['description']}'. It is recommended to wait until the rain stops before planting."
    elif weather["temperature"] < 20:
        return f"The temperature is too low ({weather['temperature']}°C) for planting. Please wait for warmer conditions."
    else:
        return f"The weather is suitable for planting. Current conditions: {weather['description']} at {weather['temperature']}°C."

# Step 5: Parse the prompt to extract soil, crop, and location
def parse_prompt(prompt):
    # Predefined soil types and crops
    soil_types = ["black soil", "laterite soil", "alluvial soil", "red soil", "marshy soil"]
    crops = ["wheat", "rice", "jowar", "bajra", "cotton", "sugarcane", "pulses", "onion"]

    # Extract soil type
    soil_type = next((soil for soil in soil_types if soil in prompt.lower()), None)

    # Extract crop type
    crop_type = next((crop for crop in crops if crop in prompt.lower()), None)

    # Extract location
    location = None
    for sentence in prompt.split('.'):
        if "live in" in sentence.lower():
            location = sentence.lower().split("live in")[-1].strip()

    return soil_type, crop_type, location

# Step 6: Main function to integrate all steps
def process_prompt(prompt):
    # Get soil-crop mapping
    soil_crop_mapping = get_soil_crop_mapping()

    # Parse the prompt
    soil_type, crop_type, location = parse_prompt(prompt)

    if not soil_type or not crop_type or not location:
        return "Invalid prompt! Please provide soil type, crop type, and location in your prompt."

    # Validate soil-crop compatibility
    is_valid, suitable_crops, response = validate_soil_crop(soil_type, crop_type, soil_crop_mapping)

    # Fetch real-time weather data if validation succeeds
    if is_valid:
        weather = get_weather_data(location)
        weather_advice = analyze_weather(weather)
        return f"{response}\n\n{weather_advice}"
    else:
        return response

# Input prompt directly in the code
prompt = "I have red soil and I want to plant rice. I live in Pune ."

# Process the prompt and display the result
result = process_prompt(prompt)
print(result)

The crop 'rice' is suitable for red soil. Here are some other crops suitable for red soil: rice.

The weather is suitable for planting. Current conditions: broken clouds at 25.85°C.
